In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# SmartRetail AI — BigQuery AI Demo\n",
    "\n",
    "This notebook demonstrates how to run BigQuery AI SQL (AI.FORECAST, AI.GENERATE_TABLE, ML.GENERATE_TEXT) from Python, save results, and preview outputs. Replace placeholders in `src/python/config.py` before running."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Install dependencies (uncomment if running on a fresh environment)\n",
    "# !pip install google-cloud-bigquery pandas\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import os\n",
    "from google.cloud import bigquery\n",
    "import pandas as pd\n",
    "from pathlib import Path\n",
    "\n",
    "# Load config (expects src/python/config.py to be configured)\n",
    "import sys\n",
    "sys.path.append(str(Path.cwd() / 'src' / 'python'))\n",
    "try:\n",
    "    import config\n",
    "except Exception as e:\n",
    "    print('Could not import config module. Make sure src/python/config.py exists and has PROJECT_ID & BQ_DATASET set.')\n",
    "    raise\n",
    "\n",
    "PROJECT_ID = config.PROJECT_ID\n",
    "BQ_DATASET = config.BQ_DATASET\n",
    "client = bigquery.Client(project=PROJECT_ID)\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1) Run Forecast (AI.FORECAST)\n",
    "\n",
    "This cell reads `src/queries/forecast.sql`, substitutes placeholders and executes the query."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "sql_path = Path('src/queries/forecast.sql')\n",
    "if not sql_path.exists():\n",
    "    raise FileNotFoundError(f'{sql_path} not found — ensure you have the query file in src/queries')\n",
    "\n",
    "raw_sql = sql_path.read_text()\n",
    "sql = raw_sql.replace('PROJECT.DATASET', f'{PROJECT_ID}.{BQ_DATASET}')\n",
    "print('Submitting forecast query...')\n",
    "job = client.query(sql)\n",
    "df_forecast = job.result().to_dataframe()\n",
    "df_forecast.head()\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2) Save Forecasts\n",
    "Save the forecast output into `results/forecasts.csv`."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "Path('results').mkdir(parents=True, exist_ok=True)\n",
    "df_forecast.to_csv('results/forecasts.csv', index=False)\n",
    "print('Saved results/forecasts.csv — rows:', len(df_forecast))\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3) Generate Insights (AI.GENERATE_TABLE)\n",
    "Run the summarization/insights query and save results."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "sql_path = Path('src/queries/generate_table.sql')\n",
    "raw_sql = sql_path.read_text()\n",
    "sql = raw_sql.replace('PROJECT.DATASET', f'{PROJECT_ID}.{BQ_DATASET}')\n",
    "print('Submitting insights query...')\n",
    "job = client.query(sql)\n",
    "df_insights = job.result().to_dataframe()\n",
    "df_insights.to_csv('results/insights.csv', index=False)\n",
    "print('Saved results/insights.csv — rows:', len(df_insights))\n",
    "df_insights.head()\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4) Generate Personalized Emails (ML.GENERATE_TEXT)\n",
    "Run the personalize SQL and save sample outputs."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "sql_path = Path('src/queries/personalize.sql')\n",
    "raw_sql = sql_path.read_text()\n",
    "sql = raw_sql.replace('PROJECT.DATASET', f'{PROJECT_ID}.{BQ_DATASET}')\n",
    "print('Submitting personalize query...')\n",
    "job = client.query(sql)\n",
    "df_emails = job.result().to_dataframe()\n",
    "df_emails.to_csv('results/personalized_emails.csv', index=False)\n",
    "print('Saved results/personalized_emails.csv — rows:', len(df_emails))\n",
    "df_emails.head()\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5) Quick display\n",
    "Show a few rows from each results file."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "for f in ['results/forecasts.csv','results/insights.csv','results/personalized_emails.csv']:\n",
    "    if Path(f).exists():\n",
    "        print('\\n==', f)\n",
    "        display(pd.read_csv(f).head())\n",
    "    else:\n",
    "        print('\\n==', f, 'not found — run previous cells')\n"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.10"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}
